Problem 3: 
How long distance individuals have travelled? (8 points)
In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the userid -column and the points created in the previous problem. You will need the shapefile Kruger_posts.shp generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:

What was the shortest distance travelled in meters?
What was the mean distance travelled in meters?
What was the maximum distance travelled in meters?
In your code, you should first:

- Import required modules.

- Read in the shapefile as a geodataframe called data

- Reproject the data from WGS84 projection into EPSG:32735 

- projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into the metric system.

- Store the result in a variable called data!

In [2]:
#Import required modules.
import geopandas as gpd
from shapely.geometry import Point, LineString
import pandas as pd
from pyproj import CRS

In [3]:
#Read in the shapefile as a geodataframe called data
fp = 'Kruger_posts.shp'
data = gpd.read_file(fp)

print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                     geometry  
0  POINT (31.48463 -24.98079)  
1  POINT (31.50891 -25.49922)  
2  POINT (30.93087 -24.34258)  
3  POINT (31.51972 -24.85461)  
4  POINT (31.52084 -24.92107)  


In [4]:
#Let’s check the data type:
type(data)

geopandas.geodataframe.GeoDataFrame

In [5]:
#Reproject the data from WGS84 projection into EPSG:32735 
data = data.to_crs(epsg=32735)

In [6]:
data.crs

<Projected CRS: EPSG:32735>
Name: WGS 84 / UTM zone 35S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 24°E and 30°E, southern hemisphere between 80°S and equator, onshore and offshore. Botswana. Burundi. Democratic Republic of the Congo (Zaire). Rwanda. South Africa. Tanzania. Uganda. Zambia. Zimbabwe.
- bounds: (24.0, -80.0, 30.0, 0.0)
Coordinate Operation:
- name: UTM zone 35S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                         geometry  
0  POINT (952912.890 7229683.258)  
1  POINT (953433.223 7172080.632)  
2  POINT (898955.144 7302197.408)  
3  POINT (956927.218 7243564.942)  
4  POINT (956794.955 7236187.926)  


In [8]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


Group the data by userid

In [9]:
#group the data by userid
grouped = data.groupby('userid')

#check group keys
#grouped.groups.keys()

print(grouped)
#print("Groups: ", grouped.groups) #grab dictionary of userid and associated indices
#print("Indicies: ", grouped.indices) #grab dictionary of userid and associated indicies in array format

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

Create LineString objects for each user connecting the points from oldest to latest:

Suggested steps:

- Create an empty DataFrame called movements.
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data:
    - sort the rows by timestamp
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the movements dataframe. You can also add the userid in a separate column (or use the userid as index).
- Convert movements into a GeoDataFrame (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the geometry column.
- Set the CRS of the movements GeoDataFrame as EPSG:32735

Test Code for Inplace sort for timestamp value looking at a subset of 10 userid values

In [ ]:
#extract the first 10 userid unique values 
subset_id = data["userid"].unique()[0:10]
print(subset_id)

#example of extracting the first unique value from our subset of unique values
print(subset_id[0])

print("Before Sort....\n")
#build a subset groupby geodataframe: userid: [ rows_related_to_user_id ] 
subset_groupby = {}
for user_id in subset_id:
    #get copy of geodataframe with userid
    subset_groupby[user_id] = grouped.get_group(user_id).copy()
    print(subset_groupby[user_id].head())

print("-------------")
print("After Sort....\n")

#sort userid rows by timestamp value inplace in ascending order
for user_id in subset_id:
    subset_groupby[user_id].sort_values('timestamp', ascending=True, inplace=True)
    print(subset_groupby[user_id].head())

    

More test code for sorting in place of Geodataframe

In [ ]:
#example of extracting Geodataframe from GroupByGeoDataFrame\
#
#
#return dataframe of rows with the following userid 
test = grouped.get_group(65281761).copy()
print(test)
print(type(test))
print("--------------------------")

#sort dataframe by timestamp in ascending order
test.sort_values('timestamp', ascending=True, inplace=True)
print(test)

type(grouped)

In [39]:
movements = gpd.GeoDataFrame()
movements['geometry'] = None
movements['userid'] = None

def return_lines(user_id_list, groupby_geo_data_frame):
    '''
    Takes a groupby dataframe, and unique id list. 
    Sorts dataframe by timestamp, creates lat long points for each row, and generates a linestring from the points for each userid
    '''
    
    lines = []

    for user_id in user_id_list:
        
        #get copy of geodataframe with userid
        temp = groupby_geo_data_frame.get_group(user_id).copy()

        #sort the timestamps of the copied rows with userid
        temp.sort_values('timestamp', ascending=True, inplace=True)

        #reset the previous index to start from 0 to N, old index is added to geodataframe as a column
        temp.reset_index(inplace=True)

        #store all points for a given userid
        points = []

        #create point with each lat, long
        #append point to line list
        for index in range(0,len(temp)):
            points.append( Point( temp.at[index,'lon'], temp.at[index,'lat'] ) )

        
        #print("userid: ",user_id , ", # of Points:", len(points))
        
        #if userid contains 1 point, create a Point object, otherwise create a LineString object
        if (len(points)> 1):
            lines.append(LineString(points))
        else:
            #make a 'LineString with the same point (point that points to itself)
            lines.append(LineString( [ points[0],points[0] ] ))
            
    return lines


#Iterate over the grouped dictionary : userid : [rows]
#print(grouped.get_group(65281761))

#get a sample of unique user ids
#all_unique_id = data["userid"].unique()[0:5]

#get all unique user_ids
all_unique_id = data["userid"].unique()

#append userid to geodataframe
movements['userid'] = all_unique_id

#append geometry to geodataframe
movements['geometry'] = return_lines(all_unique_id,grouped)

#convert movements epsg to 32735
movements.crs= CRS.from_epsg(32735).to_wkt()

print(movements.head())

                                            geometry    userid
0        LINESTRING (31.485 -24.981, 31.485 -24.981)  66487960
1  LINESTRING (31.450 -25.427, 31.509 -25.499, 31...  65281761
2  LINESTRING (30.931 -24.343, 30.931 -24.343, 30...  90916112
3  LINESTRING (31.522 -24.921, 31.521 -24.921, 31...  37959089
4  LINESTRING (31.401 -24.786, 31.402 -24.860, 31...  27793716


In [40]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

<class 'geopandas.geodataframe.GeoDataFrame'>
PROJCRS["WGS 84 / UTM zone 35S",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 35S",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",27,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",10000000,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["Engineering survey, topographic mapping."],AREA["Between 24°E and 30°E, southern hemisphere b

Finally:

- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)

- Calculate the lenghts of the lines into a new column called distance in movements GeoDataFrame.

In [20]:
print(movements.crs)

PROJCRS["WGS 84 / UTM zone 35S",BASEGEOGCRS["WGS 84",DATUM["World Geodetic System 1984",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 35S",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",27,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False easting",500000,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",10000000,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1]],USAGE[SCOPE["Engineering survey, topographic mapping."],AREA["Between 24°E and 30°E, southern hemisphere between 80°S and equator, onshore and offshore.

In [41]:
#get the distance of each linestring of the geopandas dataframe
movements['distance'] = movements.length

In [42]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,geometry,userid,distance
0,"LINESTRING (31.485 -24.981, 31.485 -24.981)",66487960,0.000000
1,"LINESTRING (31.450 -25.427, 31.509 -25.499, 31...",65281761,1.633636
2,"LINESTRING (30.931 -24.343, 30.931 -24.343, 30...",90916112,0.044455
3,"LINESTRING (31.522 -24.921, 31.521 -24.921, 31...",37959089,7.854558
4,"LINESTRING (31.401 -24.786, 31.402 -24.860, 31...",27793716,0.207887


You should now be able to print answers to the following questions:

- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

In [43]:
#get the min distance travelled
print("min: ", movements['distance'].min() )

#get the mean distance travelled
print("mean: ", movements['distance'].sum() / len(movements['distance']) )

#get the max distance travelled
print("max: ", movements['distance'].max() )

min:  0.0
mean:  0.604835107294859
max:  63.94749765236673


Finally, save the movements of into a Shapefile called some_movements.shp

In [44]:
fp = "some_movements.shp"

#each shapefile can only contain one geometry, points and lines geometry cannot be mixed together
movements.to_file(fp)

C:\Users\aly\AppData\Local\Continuum\anaconda3\envs\python-gis\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [45]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."